<a href="https://colab.research.google.com/github/kragunleo/CustomFee-Supplychain/blob/main/Audirvisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# If you haven't uploaded the file to Colab yet:
# from google.colab import files
# uploaded = files.upload()

In [9]:
from google.colab import files
import pandas as pd
import io
import numpy as np
import re

# 1. Upload the Excel file
print("Please upload your Excel file (e.g., 'AnalyseTATI-S_Modifie.xlsx'):")
uploaded = files.upload()

# 2. Extract the filename and load the specific sheet
file_name = list(uploaded.keys())[0]

# We specify sheet_name='TransInter' as it contains the audit data
# If your sheet name is different, change it here
try:
    df_raw = pd.read_excel(io.BytesIO(uploaded[file_name]), sheet_name='TransInter')
except:
    # Fallback: if sheet name is unknown, load the first sheet
    df_raw = pd.read_excel(io.BytesIO(uploaded[file_name]), sheet_name=0)

# 3. Clean the Header Structure
# If row 0 contains the headers like 'Année', 'N° Dos.', etc.
df = df_raw.copy()
if 'Année' not in df.columns:
    df.columns = df.iloc[0]
    df = df.drop(0).reset_index(drop=True)

# 4. Helper function to clean currency/text and convert to float
def clean_currency(value):
    if pd.isna(value) or str(value).strip().lower() == 'nan':
        return 0.0
    # Convert to string and strip spaces
    clean_val = str(value).replace(' ', '').replace(',', '.')
    # Regex to keep only digits and the first decimal point
    clean_val = re.sub(r'[^\d.]', '', clean_val)
    try:
        return float(clean_val)
    except:
        return 0.0

# 5. Define and Clean Audit Columns
cols_to_fix = [
    'Droit &Taxe',
    'Honoraires facturés par le commissionnaire',
    'Honoraires calculés selon FEDERMAR',
    'Ecart Facturé vs Theorique',
    'Valeur Statisque/ Valeur CAF'
]

for col in cols_to_fix:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)

# 6. Add Audit Metrics
df['Overbilling_Ratio'] = np.where(
    df['Honoraires calculés selon FEDERMAR'] > 0,
    (df['Ecart Facturé vs Theorique'] / df['Honoraires calculés selon FEDERMAR']) * 100,
    0
)

df['Audit_Risk_Level'] = df['Overbilling_Ratio'].apply(
    lambda x: '🚨 Critical' if x > 20 else ('⚠️ Warning' if x > 5 else '✅ Compliant')
)

print(f"\n✅ Successfully loaded: {file_name}")
print(f"📊 Total Dossiers Analyzed: {len(df)}")
df[['N° Dos.', 'Droit &Taxe', 'Honoraires facturés par le commissionnaire', 'Audit_Risk_Level']].head()

Please upload your Excel file (e.g., 'AnalyseTATI-S_Modifie.xlsx'):


Saving Analyse table for the code.xlsx to Analyse table for the code (2).xlsx

✅ Successfully loaded: Analyse table for the code (2).xlsx
📊 Total Dossiers Analyzed: 251


,N° Dos.,Droit &Taxe,Honoraires facturés par le commissionnaire,Audit_Risk_Level
0,130 22 0906,43530.0,74629.0,✅ Compliant
1,130 22 0929,0.0,10500000.0,🚨 Critical
2,139 22 0924,101099.0,2283989.0,🚨 Critical
3,IX 230012,0.0,450000.0,🚨 Critical
4,130 23 1343,543425.0,5370476.0,🚨 Critical


In [10]:
import plotly.express as px
import plotly.graph_objects as go

# 1. Prepare Data for Outlier Analysis
# We sort by the highest gap to show where the most money is being lost
top_discrepancies = df.sort_values(by='Ecart Facturé vs Theorique', ascending=False).head(10)

# 2. Visualizing Top 10 Overbilled Dossiers
fig1 = px.bar(
    top_discrepancies,
    x='N° Dos.',
    y='Ecart Facturé vs Theorique',
    color='Audit_Risk_Level',
    title='Top 10 Financial Discrepancies (Potential Overbilling)',
    labels={'Ecart Facturé vs Theorique': 'Discrepancy Amount (FCFA)'},
    color_discrete_map={'🚨 Critical': '#d62728', '⚠️ Warning': '#ff7f0e', '✅ Compliant': '#2ca02c'},
    template='plotly_white'
)

# Improve layout
fig1.update_layout(xaxis_tickangle=-45)
fig1.show()

# 3. Audit Risk Distribution (Portfolio Health)
# This shows what percentage of your total operations are risky
risk_counts = df['Audit_Risk_Level'].value_counts().reset_index()
risk_counts.columns = ['Risk Level', 'Count']

fig2 = px.pie(
    risk_counts,
    values='Count',
    names='Risk Level',
    hole=0.4,
    title='Overall Audit Risk Distribution',
    color='Risk Level',
    color_discrete_map={'🚨 Critical': '#d62728', '⚠️ Warning': '#ff7f0e', '✅ Compliant': '#2ca02c'}
)
fig2.update_traces(textinfo='percent+label')
fig2.show()

# 4. Efficiency Analysis: Days vs. Cost (Optional but powerful)
# This chart checks if "Longer processing time" correlates with "Higher Fees"
# Note: You can change 'Droit &Taxe' to 'Valeur Statisque/ Valeur CAF' if preferred
fig3 = px.scatter(
    df[df['Honoraires facturés par le commissionnaire'] > 0],
    x='Droit &Taxe',
    y='Honoraires facturés par le commissionnaire',
    size='Overbilling_Ratio',
    color='Audit_Risk_Level',
    hover_name='N° Dos.',
    log_x=True, # Helps deal with wide ranges in tax amounts
    title='Relationship: Tax Volume vs. Commission Fees',
    labels={'Droit &Taxe': 'Total Taxes (FCFA)', 'Honoraires facturés par le commissionnaire': 'Invoiced Fee'}
)
fig3.show()

In [13]:
# 1. Filter for Actionable Items
# We focus on 'Critical' and 'Warning' levels for the final report
report_df = df[df['Audit_Risk_Level'] != '✅ Compliant'].copy()

# 2. Calculate Financial Impact
total_recoverable = report_df['Ecart Facturé vs Theorique'].sum()
avg_overcharge = report_df['Overbilling_Ratio'].mean()
high_risk_count = len(df[df['Audit_Risk_Level'] == '🚨 Critical'])

# 3. Generate Executive Summary Text
print("="*50)
print("AUDIT EXECUTIVE SUMMARY")
print("="*50)
print(f"Total Dossiers Scanned:      {len(df)}")
print(f"Dossiers with Discrepancies: {len(report_df)}")
print(f"Critical Risk Level Files:   {high_risk_count}")
print(f"Potential Savings/Recovery:  {total_recoverable:,.0f} FCFA")
print(f"Average Overbilling Rate:    {avg_overcharge:.2f}%")
print("="*50)

# 4. Highlight the Top 5 "Audit Observations"
# Assuming 'Column51' or 'Commentaire' is the column with the auditor's notes
# If your column name is different, update it here
comment_col = 'Commentaire' if 'Commentaire' in df.columns else 'Column51'

print("\nTOP 5 CRITICAL DOSSIERS & OBSERVATIONS:")
top_5_critical = report_df.sort_values(by='Ecart Facturé vs Theorique', ascending=False).head(5)

for i, row in top_5_critical.iterrows():
    print(f"\n📌 Dossier: {row['N° Dos.']} | Gap: {row['Ecart Facturé vs Theorique']:,.0f} FCFA")
    print(f"   Observation: {row[comment_col]}")

# 5. Export Findings to Excel for Management
# This creates a professional 'To-Action' list
output_filename = "Audit_Action_Plan.xlsx"
report_df.to_excel(output_filename, index=False)

from google.colab import files
files.download(output_filename)

print(f"\n✅ Professional Audit Report exported as: {output_filename}")

AUDIT EXECUTIVE SUMMARY
Total Dossiers Scanned:      251
Dossiers with Discrepancies: 184
Critical Risk Level Files:   182
Potential Savings/Recovery:  1,100,012,038 FCFA
Average Overbilling Rate:    1290.89%

TOP 5 CRITICAL DOSSIERS & OBSERVATIONS:

📌 Dossier: 130 23 1230 | Gap: 42,006,024 FCFA
   Observation: Facturation de trop.

📌 Dossier: 130 25 2365 | Gap: 31,229,278 FCFA
   Observation: Les rubriques intervention douane colis urgent 9350000Fcfa,Transfert de document 7508160FCFA et TS en douane VAD 60000fcfa Surfacture.HAD selon FEDERMAR est surfacture.Abscence de document declaratif

📌 Dossier: 130 25 2239 | Gap: 31,202,384 FCFA
   Observation: Les rubriques Intervention douanes 8500000, transfert de document 1887105,ts douane VAD 60 000 sont surfacture.Magazinage 3578500 facture sans justificatif.HAD selon FEDERMAR surfacture.Abscence de document declaratifs

📌 Dossier: 130 25 2248 | Gap: 28,322,308 FCFA
   Observation: Les rubriques Intervention douane 1975000FCFA, Transfert d

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Professional Audit Report exported as: Audit_Action_Plan.xlsx
